# Prompt Engineering Fundamentals 

---
In this demo notebook, we are going to explore some of the Prompt Engineering Techniques for better model output generation.

We demonstrate how to use the boto3 Python SDK to work with *Amazon Bedrock Foundation Models*  We will be using the below models in this notebook;

* Amazon Nova Lite v1.0
* Amazon Nova Micro v1.0
* Anthropic Claude Haiku v3.5
---

In [1]:
# install necessary libraries
import sys
import os
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils.environment_validation import validate_environment, validate_model_access
validate_environment()

Validating base environment
Base environment validated successfully


Validating lab environment from requirements.txt ✨

ENVIRONMENT STATUS
✅  boto3>=1.37.22, <2.0.0 is installed
✅  botocore>=1.37.22, <2.0.0 is installed

All required libraries are installed.🎉
You may proceed with the lab! 🚀

In [2]:
# If the below show that you do not have these model accesses, please go to the Bedrock console to get access.
required_models = [
    "amazon.nova-lite-v1:0",
    "amazon.nova-micro-v1:0",
    "anthropic.claude-3-5-haiku-20241022-v1:0"
]
validate_model_access(required_models)

MODEL ACCESS STATUS
✅  amazon.nova-lite-v1:0 is accessible
✅  amazon.nova-micro-v1:0 is accessible
✅  anthropic.claude-3-5-haiku-20241022-v1:0 is accessible

All required models are accessible.🎉
You may proceed with the lab! 🚀

In [3]:
# rich for pretty printing
from rich import print as rprint
from rich.markdown import Markdown

# boto3 for AWS SDK
import boto3


In [4]:
# as with other AWS services Bedrock exists in Control Plane (CP) and Data Plane (Runtime) actions.
bedrock_cp = boto3.client("bedrock")
bedrock_runtime = boto3.client("bedrock-runtime")

#### Validate the connection

We can check the client works by trying out the `list_foundation_models()` method, which will tell us all the models available for us to use.  These are models that are available, and not necessarily the models that you have access to.

In [5]:
foundation_models = bedrock_cp.list_foundation_models()
rprint(foundation_models)

{
    'ResponseMetadata': {
        'RequestId': '6f4827a7-e6a1-4de7-9e92-fc8815acc392',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'date': 'Mon, 12 May 2025 09:03:48 GMT',
            'content-type': 'application/json',
            'content-length': '57809',
            'connection': 'keep-alive',
            'x-amzn-requestid': '6f4827a7-e6a1-4de7-9e92-fc8815acc392'
        },
        'RetryAttempts': 0
    },
    'modelSummaries': [
        {
            'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large',
            'modelId': 'amazon.titan-tg1-large',
            'modelName': 'Titan Text Large',
            'providerName': 'Amazon',
            'inputModalities': ['TEXT'],
            'outputModalities': ['TEXT'],
            'responseStreamingSupported': True,
            'customizationsSupported': [],
            'inferenceTypesSupported': ['ON_DEMAND'],
            'modelLifecycle': {'status': 'ACTIVE'}
        },
        {
            'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.nova-premier-v1:0:8k',
            'modelId': 'amazon.nova-premier-v1:0:8k',
            'modelName': 'Nova Premier',
            'providerName': 'Amazon',
            'inputModalities': ['TEXT', 'IMAGE', 'VIDEO'],
            'outputModalities': ['TEXT'],
            'responseStreamingSupported': True,
            'customizationsSupported': [],
            'inferenceTypesSupported': [],
            'modelLifecycle': {'status': 'ACTIVE'}
        },
        {
            'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.nova-premier-v1:0:20k',
            'modelId': 'amazon.nova-premier-v1:0:20k',
            'modelName': 'Nova Premier',
            'providerName': 'Amazon',
            'inputModalities': ['TEXT', 'IMAGE', 'VIDEO'],
            'outputModalities': ['TEXT'],
            'responseStreamingSupported': True,
            'customizationsSupported': [],
            'inferenceTypesSupported': [],
            'modelLifecycle': {'status': 'ACTIVE'}
        },
        {
            'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.nova-premier-v1:0:1000k',
            'modelId': 'amazon.nova-premier-v1:0:1000k',
            'modelName': 'Nova Premier',
            'providerName': 'Amazon',
            'inputModalities': ['TEXT', 'IMAGE', 'VIDEO'],
            'outputModalities': ['TEXT'],
            'responseStreamingSupported': True,
            'customizationsSupported': [],
            'inferenceTypesSupported': [],
            'modelLifecycle': {'status': 'ACTIVE'}
        },
        {
            'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.nova-premier-v1:0:mm',
            'modelId': 'amazon.nova-premier-v1:0:mm',
            'modelName': 'Nova Premier',
            'providerName': 'Amazon',
            'inputModalities': ['TEXT', 'IMAGE', 'VIDEO'],
            'outputModalities': ['TEXT'],
            'responseStreamingSupported': True,
            'customizationsSupported': [],
            'inferenceTypesSupported': [],
            'modelLifecycle': {'status': 'ACTIVE'}
        },
        {
            'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.nova-premier-v1:0',
            'modelId': 'amazon.nova-premier-v1:0',
            'modelName': 'Nova Premier',
            'providerName': 'Amazon',
            'inputModalities': ['TEXT', 'IMAGE', 'VIDEO'],
            'outputModalities': ['TEXT'],
            'responseStreamingSupported': True,
            'customizationsSupported': [],
            'inferenceTypesSupported': ['INFERENCE_PROFILE'],
            'modelLifecycle': {'status': 'ACTIVE'}
        },
        {
            'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-g1-text-02',
            'modelId': 'amazon.titan-embed-g1-text-02',
            'modelName': 'Titan Text Embeddings v2',
            'providerName': 'Amazon',
            'inputModalities': ['TE

---

## The Bedrock `Converse` API

Bedrock's [`Converse` API](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html) simplifies the calling of LLMs, providing a uniform interface across the different models. The `Converse` API offers;

* Standardized inputs and outputs across all models
* Better conversation handling via the standardize message format
* Access to additional functionality such as tool_usage

We will be using the below `Converse` API call in this notebook.

```
   response = bedrock_runtime.converse(
        modelId=model_id, # or inference_profile id
        inferenceConfig=inference_configs,
        system=system_prompts,
        messages=messages
    )
```

Let's build each of the above parameters below.

---
### `modelId` and Inference Profiles
This field indicates the model to be used.   E.g., for Amazon Nova Micro 1.0, it'd be `amazon.nova-micro-v1:0`

The `modelId` field can take either a model id or an inference profile id. [Inference profile](https://docs.aws.amazon.com/bedrock/latest/userguide/inference-profiles.html), allows for automatic routing of model invocations across multiple regions. With this capability, you can burst capacity across the different regions, seamlessly.  You can [create your own inference profiles](https://docs.aws.amazon.com/bedrock/latest/userguide/inference-profiles-create.html) to track model usage across multiple users, projects, lines of business, etc.

We will be using Amazon managed inference profiles in this notebook.


In [6]:
inf_profiles = bedrock_cp.list_inference_profiles()
rprint(inf_profiles)

{
    'ResponseMetadata': {
        'RequestId': '66d8c337-e2c3-4792-a53f-a5e44b178cf3',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'date': 'Mon, 12 May 2025 09:04:03 GMT',
            'content-type': 'application/json',
            'content-length': '16577',
            'connection': 'keep-alive',
            'x-amzn-requestid': '66d8c337-e2c3-4792-a53f-a5e44b178cf3'
        },
        'RetryAttempts': 0
    },
    'inferenceProfileSummaries': [
        {
            'inferenceProfileName': 'US Anthropic Claude 3 Haiku',
            'description': 'Routes requests to Anthropic Claude 3 Haiku in us-east-1 and us-west-2.',
            'createdAt': datetime.datetime(2024, 8, 26, 0, 0, tzinfo=tzlocal()),
            'updatedAt': datetime.datetime(2024, 8, 26, 0, 0, tzinfo=tzlocal()),
            'inferenceProfileArn': 
'arn:aws:bedrock:us-west-2:444027413111:inference-profile/us.anthropic.claude-3-haiku-20240307-v1:0',
            'models': [
                {
                    'modelArn': 
'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-haiku-20240307-v1:0'
                },
                {
                    'modelArn': 
'arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-haiku-20240307-v1:0'
                }
            ],
            'inferenceProfileId': 'us.anthropic.claude-3-haiku-20240307-v1:0',
            'status': 'ACTIVE',
            'type': 'SYSTEM_DEFINED'
        },
        {
            'inferenceProfileName': 'US Anthropic Claude 3.5 Sonnet',
            'description': 'Routes requests to Anthropic Claude 3.5 Sonnet in us-east-1 and us-west-2.',
            'createdAt': datetime.datetime(2024, 8, 26, 0, 0, tzinfo=tzlocal()),
            'updatedAt': datetime.datetime(2024, 8, 26, 0, 0, tzinfo=tzlocal()),
            'inferenceProfileArn': 
'arn:aws:bedrock:us-west-2:444027413111:inference-profile/us.anthropic.claude-3-5-sonnet-20240620-v1:0',
            'models': [
                {
                    'modelArn': 
'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-5-sonnet-20240620-v1:0'
                },
                {
                    'modelArn': 
'arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-5-sonnet-20240620-v1:0'
                }
            ],
            'inferenceProfileId': 'us.anthropic.claude-3-5-sonnet-20240620-v1:0',
            'status': 'ACTIVE',
            'type': 'SYSTEM_DEFINED'
        },
        {
            'inferenceProfileName': 'US Anthropic Claude 3 Sonnet',
            'description': 'Routes requests to Anthropic Claude 3 Sonnet in us-east-1 and us-west-2.',
            'createdAt': datetime.datetime(2024, 8, 26, 0, 0, tzinfo=tzlocal()),
            'updatedAt': datetime.datetime(2024, 8, 26, 0, 0, tzinfo=tzlocal()),
            'inferenceProfileArn': 
'arn:aws:bedrock:us-west-2:444027413111:inference-profile/us.anthropic.claude-3-sonnet-20240229-v1:0',
            'models': [
                {
                    'modelArn': 
'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
                },
                {
                    'modelArn': 
'arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
                }
            ],
            'inferenceProfileId': 'us.anthropic.claude-3-sonnet-20240229-v1:0',
            'status': 'ACTIVE',
            'type': 'SYSTEM_DEFINED'
        },
        {
            'inferenceProfileName': 'US Anthropic Claude 3 Opus',
            'description': 'Routes requests to Anthropic Cluade 3 Opus in us-east-1 and us-west-2.',
            'createdAt': datetime.datetime(2024, 8, 26, 0, 0, tzinfo=tzlocal()),
            'updatedAt': datetime.datetime(2024, 8, 26, 0, 0, tzinfo=tzlocal()),
            'inferenceProfileArn': 
'arn:aws:bedrock:us-west-2:444027413111:inference-profile/us.anthropic.claude-3-opus-20240229-v1:0',
            'models': [
                {'modelArn': 'arn:aws:bedr

---
### Inference Profiles
Let's store the Amazon managed inference profiles for each of our models in individual variables.  We will use these variables in the `modelId` field of the `Converse` API call.

```python
modelId = inf_profile_nova_micro_v1
```

In [7]:
for inf_prof in inf_profiles['inferenceProfileSummaries']:
    if inf_prof['inferenceProfileName'] == 'US Nova Micro':
        rprint(inf_prof)
        inf_profile_nova_micro_v1 = inf_prof['inferenceProfileId']

    if inf_prof['inferenceProfileName'] == 'US Nova Lite':
        rprint(inf_prof)
        inf_profile_nova_lite_v1 = inf_prof['inferenceProfileId']

    if inf_prof['inferenceProfileName'] == 'US Anthropic Claude 3.5 Haiku':
        rprint(inf_prof)
        inf_profile_claude_haiku_v3_5 = inf_prof['inferenceProfileId']
        

rprint("This the lightest Amazon Nova LLM.  We will be using this model in most examples of this notebook.\n", inf_profile_nova_micro_v1, "\n")
rprint("This is the smallest model in the Anthropic family of models", inf_profile_claude_haiku_v3_5, "\n")
rprint("This is the midrange Amazon Nova LLM.  We will use it in the multi-modal use case.\n", inf_profile_nova_lite_v1, "\n")

{
    'inferenceProfileName': 'US Anthropic Claude 3.5 Haiku',
    'description': 'Routes requests to Anthropic Claude 3.5 Haiku in us-east-1, us-east-2 and us-west-2.',
    'createdAt': datetime.datetime(2024, 10, 22, 0, 0, tzinfo=tzlocal()),
    'updatedAt': datetime.datetime(2024, 10, 22, 0, 0, tzinfo=tzlocal()),
    'inferenceProfileArn': 
'arn:aws:bedrock:us-west-2:444027413111:inference-profile/us.anthropic.claude-3-5-haiku-20241022-v1:0',
    'models': [
        {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-5-haiku-20241022-v1:0'},
        {'modelArn': 'arn:aws:bedrock:us-east-2::foundation-model/anthropic.claude-3-5-haiku-20241022-v1:0'},
        {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-5-haiku-20241022-v1:0'}
    ],
    'inferenceProfileId': 'us.anthropic.claude-3-5-haiku-20241022-v1:0',
    'status': 'ACTIVE',
    'type': 'SYSTEM_DEFINED'
}

{
    'inferenceProfileName': 'US Nova Lite',
    'description': 'Routes requests to Nova Lite in us-east-1, us-west-2 and us-east-2.',
    'createdAt': datetime.datetime(2024, 11, 29, 13, 23, tzinfo=tzlocal()),
    'updatedAt': datetime.datetime(2024, 11, 29, 13, 23, tzinfo=tzlocal()),
    'inferenceProfileArn': 'arn:aws:bedrock:us-west-2:444027413111:inference-profile/us.amazon.nova-lite-v1:0',
    'models': [
        {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-lite-v1:0'},
        {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.nova-lite-v1:0'},
        {'modelArn': 'arn:aws:bedrock:us-east-2::foundation-model/amazon.nova-lite-v1:0'}
    ],
    'inferenceProfileId': 'us.amazon.nova-lite-v1:0',
    'status': 'ACTIVE',
    'type': 'SYSTEM_DEFINED'
}

{
    'inferenceProfileName': 'US Nova Micro',
    'description': 'Routes requests to Nova Micro in us-east-1, us-west-2 and us-east-2.',
    'createdAt': datetime.datetime(2024, 11, 29, 13, 23, tzinfo=tzlocal()),
    'updatedAt': datetime.datetime(2024, 11, 29, 13, 23, tzinfo=tzlocal()),
    'inferenceProfileArn': 'arn:aws:bedrock:us-west-2:444027413111:inference-profile/us.amazon.nova-micro-v1:0',
    'models': [
        {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-micro-v1:0'},
        {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.nova-micro-v1:0'},
        {'modelArn': 'arn:aws:bedrock:us-east-2::foundation-model/amazon.nova-micro-v1:0'}
    ],
    'inferenceProfileId': 'us.amazon.nova-micro-v1:0',
    'status': 'ACTIVE',
    'type': 'SYSTEM_DEFINED'
}

This the lightest Amazon Nova LLM.  We will be using this model in most examples of this notebook.
 us.amazon.nova-micro-v1:0

This is the smallest model in the Anthropic family of models us.anthropic.claude-3-5-haiku-20241022-v1:0

This is the midrange Amazon Nova LLM.  We will use it in the multi-modal use case.
 us.amazon.nova-lite-v1:0

---
### `inferenceConfig`
Inference parameters allow you to control the behavior of the model.  The `inferenceConfig` field is a dictionary that can contain the following parameters:

temperature (0.0 to 1.0, default 0.7):

- Lower values (like 0.0) make responses more deterministic although this is not guaranteed
- With a value of 0.0, the model will choose the most likely next token, higher values (like 1.0) will make the model choose other less likely tokens
- Higher values (like 1.0) make responses more randomized

maxTokens (integer, default 5k):
- Controls the maximum length of the model's response
- Model's response may get truncated if it exceeds this value

topP (0.0 to 1.0, default 0.9):
- Each token is sampled based on its probability of being the next token given the previous tokens
- The model will only consider the top P% cumulative probability mass for the next token
   
Note that not all models support all inference parameters



In [14]:
inference_configs = {
    "temperature": 0.5,        # Controls randomness (0.0 to 1.0)
    "maxTokens": 2048,         # Maximum tokens in the response
    "topP": 0.9,               # Sample from the top 90% of the probability mass

}
rprint(inference_configs)

{'temperature': 0.5, 'maxTokens': 2048, 'topP': 0.9}

---
### `system` Prompts

System prompts are your (the developer's) directive to the LLM, as opposed to the user's.  It is useful for setting the pretext of the LLM task and defining the role of the LLM.  For example, you can set the system prompt to be a "helpful assistant" or "a sarcastic assistant".  The system prompt is not visible to the end user.





In [15]:
system_prompts = [{"text": "You are a financial advisor of a large asset management firm."}]

---
#### `messages`

Messages are the conversation history between the user and the LLM.  The messages are a list of dictionaries, where each dictionary contains the following fields:
- `role`: The role of the message sender.  This can be either `user` or `assistant`
- `content`: The content of the message.  This can include text, images, files, etc.

Every component of the interaction is recorded as a list of messages.


In [16]:
messages = [] # start with an empty convesation
user_msg1 = "What is the best way to save for my children's college tuitions?"

message_1 = {
    "role": "user",
    "content": [{"text": user_msg1}]
}

messages.append(message_1)  # later append outputs for a conversation

---

## Let's now call Bedrock!

Let's first try with Amazon Nova Micro v1

In [17]:
response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_configs
)

In [18]:
rprint(response)
rprint(response['output']['message']['content'][0]['text'])    

{
    'ResponseMetadata': {
        'RequestId': '0a938c70-0ed2-48b6-bc19-71ea9a948c05',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'date': 'Mon, 12 May 2025 09:22:02 GMT',
            'content-type': 'application/json',
            'content-length': '3559',
            'connection': 'keep-alive',
            'x-amzn-requestid': '0a938c70-0ed2-48b6-bc19-71ea9a948c05'
        },
        'RetryAttempts': 1
    },
    'output': {
        'message': {
            'role': 'assistant',
            'content': [
                {
                    'text': "Saving for your children's college tuition is a significant financial goal that 
requires careful planning and strategy. Here are some of the best ways to save effectively:\n\n### 1. **Start 
Early**\n- **Time Advantage**: The earlier you start saving, the more time your investments have to grow through 
compound interest.\n- **Regular Contributions**: Even small, regular contributions can accumulate significantly 
over time.\n\n### 2. **Utilize Tax-Advantaged Accounts**\n- **529 Plans**: These are tax-advantaged savings plans 
specifically designed for education expenses. Contributions grow tax-free, and withdrawals for qualified education 
expenses are also tax-free.\n  - **Types**: There are two main types: prepaid tuition plans and education savings 
plans.\n- **Coverdell Education Savings Account (ESA)**: Another tax-advantaged account with lower contribution 
limits but broader investment options.\n- **Roth IRA**: While primarily for retirement, Roth IRAs can be used for 
education expenses without penalties, though you'll pay taxes on the earnings.\n\n### 3. **Invest in Low-Cost Index
Funds**\n- **Diversification**: Low-cost index funds provide broad market exposure and are less risky than 
individual stocks.\n- **Expense Ratios**: Lower expense ratios mean more of your money works for you.\n\n### 4. 
**Employer-Sponsored Plans**\n- **Employer Matching**: If your employer offers a 401(k) match, contribute at least 
enough to get the full match.\n- **Flexible Spending Accounts (FSAs)**: Some employers offer FSAs for education 
expenses, allowing you to set aside pre-tax dollars.\n\n### 5. **Scholarships and Grants**\n- **Encourage Academic 
Excellence**: Encourage your children to perform well academically, as good grades can lead to scholarships.\n- 
**Financial Aid Applications**: Help your children complete the Free Application for Federal Student Aid (FAFSA) to
determine eligibility for grants and loans.\n\n### 6. **Budgeting and Reducing Expenses**\n- **Cut Unnecessary 
Costs**: Review your household budget and identify areas where you can cut back to free up more money for college 
savings.\n- **Emergency Fund**: Ensure you have an emergency fund to avoid dipping into your college savings for 
unexpected expenses.\n\n### 7. **Consider Part-Time Work and Internships**\n- **Earnings Contribution**: If your 
child is capable, part-time work or internships can contribute to their education fund.\n- **Work-Study Programs**:
Encourage participation in college work-study programs.\n\n### 8. **Gifts and Inheritances**\n- **Gifts**: Gifts 
from family and friends can be used for education expenses without incurring gift tax, up to certain limits.\n- 
**Inheritances**: Plan your estate to include provisions for your children’s education.\n\n### 9. **Regularly 
Review and Adjust**\n- **Portfolio Rebalancing**: Regularly review your investment portfolio to ensure it aligns 
with your risk tolerance and time horizon.\n- **Adjust Contributions**: As your financial situation changes, adjust
your contributions to your savings plans accordingly.\n\n### 10. **Professional Advice**\n- **Consult a Financial 
Advisor**: A professional can provide personalized advice and help you navigate the complexities of saving for 
college.\n\nBy combining these strategies, you can create a robust plan to save for your children's college tuition
effectively."
                }


Saving for your children's college tuition is a significant financial goal that requires careful planning and 
strategy. Here are some of the best ways to save effectively:

### 1. **Start Early**
- **Time Advantage**: The earlier you start saving, the more time your investments have to grow through compound 
interest.
- **Regular Contributions**: Even small, regular contributions can accumulate significantly over time.

### 2. **Utilize Tax-Advantaged Accounts**
- **529 Plans**: These are tax-advantaged savings plans specifically designed for education expenses. Contributions
grow tax-free, and withdrawals for qualified education expenses are also tax-free.
  - **Types**: There are two main types: prepaid tuition plans and education savings plans.
- **Coverdell Education Savings Account (ESA)**: Another tax-advantaged account with lower contribution limits but 
broader investment options.
- **Roth IRA**: While primarily for retirement, Roth IRAs can be used for education expenses without penalties, 
though you'll pay taxes on the earnings.

### 3. **Invest in Low-Cost Index Funds**
- **Diversification**: Low-cost index funds provide broad market exposure and are less risky than individual 
stocks.
- **Expense Ratios**: Lower expense ratios mean more of your money works for you.

### 4. **Employer-Sponsored Plans**
- **Employer Matching**: If your employer offers a 401(k) match, contribute at least enough to get the full match.
- **Flexible Spending Accounts (FSAs)**: Some employers offer FSAs for education expenses, allowing you to set 
aside pre-tax dollars.

### 5. **Scholarships and Grants**
- **Encourage Academic Excellence**: Encourage your children to perform well academically, as good grades can lead 
to scholarships.
- **Financial Aid Applications**: Help your children complete the Free Application for Federal Student Aid (FAFSA) 
to determine eligibility for grants and loans.

### 6. **Budgeting and Reducing Expenses**
- **Cut Unnecessary Costs**: Review your household budget and identify areas where you can cut back to free up more
money for college savings.
- **Emergency Fund**: Ensure you have an emergency fund to avoid dipping into your college savings for unexpected 
expenses.

### 7. **Consider Part-Time Work and Internships**
- **Earnings Contribution**: If your child is capable, part-time work or internships can contribute to their 
education fund.
- **Work-Study Programs**: Encourage participation in college work-study programs.

### 8. **Gifts and Inheritances**
- **Gifts**: Gifts from family and friends can be used for education expenses without incurring gift tax, up to 
certain limits.
- **Inheritances**: Plan your estate to include provisions for your children’s education.

### 9. **Regularly Review and Adjust**
- **Portfolio Rebalancing**: Regularly review your investment portfolio to ensure it aligns with your risk 
tolerance and time horizon.
- **Adjust Contributions**: As your financial situation changes, adjust your contributions to your savings plans 
accordingly.

### 10. **Professional Advice**
- **Consult a Financial Advisor**: A professional can provide personalized advice and help you navigate the 
complexities of saving for college.

By combining these strategies, you can create a robust plan to save for your children's college tuition 
effectively.

---

## Bedrock Standardizes Different Model Inputs
As previously mentioned, Bedrock Converse API allows the developer to invoke different models with a standard interface.  This means that the same API call can be used to invoke different models, even if they come from different model providers.

Let's now try the same call that we did with Amazon Nova Micro v1, but with Anthropic Claude Haiku v3.5

In [19]:
response = bedrock_runtime.converse(
    modelId=inf_profile_claude_haiku_v3_5,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_configs
)

In [20]:
rprint(response)
rprint(response['output']['message']['content'][0]['text'])   

{
    'ResponseMetadata': {
        'RequestId': '70a85d1e-b20c-4989-8484-308b5dffb718',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'date': 'Mon, 12 May 2025 09:23:17 GMT',
            'content-type': 'application/json',
            'content-length': '1493',
            'connection': 'keep-alive',
            'x-amzn-requestid': '70a85d1e-b20c-4989-8484-308b5dffb718'
        },
        'RetryAttempts': 0
    },
    'output': {
        'message': {
            'role': 'assistant',
            'content': [
                {
                    'text': 'As a financial advisor, I recommend several strategies for saving for college 
tuition:\n\n1. 529 College Savings Plan\n- Tax-advantaged investment account\n- Earnings grow tax-free\n- 
Withdrawals are tax-free for qualified education expenses\n- State tax deductions in many states\n- Flexible 
transfer between family members\n\n2. Coverdell Education Savings Account (ESA)\n- Similar tax benefits to 529 
plans\n- Can be used for K-12 expenses as well\n- Lower contribution limits\n\n3. Roth IRA\n- Flexible investment 
option\n- Can withdraw contributions penalty-free\n- Can use funds for education expenses\n\n4. Strategies to 
maximize savings:\n- Start early to benefit from compound growth\n- Automatic monthly contributions\n- Invest in 
age-based or target-date funds\n- Consider a mix of investment vehicles\n- Look into prepaid tuition plans\n\n5. 
Additional tips:\n- Encourage children to apply for scholarships\n- Consider community college for first two 
years\n- Explore work-study programs\n- Investigate employer tuition reimbursement\n\nThe best approach depends on 
your specific financial situation, income, and goals. I recommend consulting a financial advisor for personalized 
advice.'
                }
            ]
        }
    },
    'stopReason': 'end_turn',
    'usage': {
        'inputTokens': 34,
        'outputTokens': 273,
        'totalTokens': 307,
        'cacheReadInputTokens': 0,
        'cacheWriteInputTokens': 0
    },
    'metrics': {'latencyMs': 5216}
}

As a financial advisor, I recommend several strategies for saving for college tuition:

1. 529 College Savings Plan
- Tax-advantaged investment account
- Earnings grow tax-free
- Withdrawals are tax-free for qualified education expenses
- State tax deductions in many states
- Flexible transfer between family members

2. Coverdell Education Savings Account (ESA)
- Similar tax benefits to 529 plans
- Can be used for K-12 expenses as well
- Lower contribution limits

3. Roth IRA
- Flexible investment option
- Can withdraw contributions penalty-free
- Can use funds for education expenses

4. Strategies to maximize savings:
- Start early to benefit from compound growth
- Automatic monthly contributions
- Invest in age-based or target-date funds
- Consider a mix of investment vehicles
- Look into prepaid tuition plans

5. Additional tips:
- Encourage children to apply for scholarships
- Consider community college for first two years
- Explore work-study programs
- Investigate employer tuition reimbursement

The best approach depends on your specific financial situation, income, and goals. I recommend consulting a 
financial advisor for personalized advice.

## Prompt Engineering - Basic Techiques

In this section, we are going to explore some fundamental prompt engineering techniques.

---

### Zero-shot prompting
Zero Shot prompting describes the technique where we present a task to an LLM without giving it further examples. We therefore, expect it to perform the task without getting a prior “shot” at the task. Hence, “zero-shot” prompting. Modern LLMs demonstrate remarkable zero-shot performance and a positive correlation can be drawn between model size and zero-shot performance.

In [21]:

# If you'd like to try your own prompt, edit this parameter!
zero_shot_system_prompt = """You are a customer service agent tasked with classifying emails by type. Please output your answer and then justify your classification. How would you categorize this email? """

zero_shot_prompt = """
<email>
I would like to know how I can contribute to my retirement account? Any additional resources to read about IRA would be helpful.
</email> 
Provide and explanation for your choice of answer

The categories are: 
(A) IRA 
(B) 529 Plan
(C) Cash Management
(D) Youth Account
"""



The xml tags above were historically used in earlier Anthropic Claude models to breakdown the prompt into logical segments. Models from other providers, and the current Anthropic Claude models 3+, do not require these xml tags. But they can still be useful in prompt construction as they allow you to reference specific parts of the prompt.  For example, you can use the `<input>` tag to reference the input text in the prompt.  This is useful for creating dynamic prompts that can change based on the input text.

In [22]:
# Let's try this with Amazon Nova Lite v1

zero_shot_msg = {
    "role": "user",
    "content": [{"text": zero_shot_prompt}]
}

zero_shot_messages = [zero_shot_msg]

response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=zero_shot_messages,
    system=[{"text": zero_shot_system_prompt}],
    inferenceConfig=inference_configs
)

rprint(response['output']['message']['content'][0]['text'])

I would categorize this email under:

(A) IRA

**Explanation:**

The email specifically mentions contributing to a "retirement account" and requests "additional resources to read 
about IRA." The term "IRA" stands for Individual Retirement Account, which is a type of retirement savings account 
designed to help individuals save for retirement with certain tax advantages. 

Here's a breakdown of the key points that lead to this classification:

1. **Retirement Account**: The email refers to contributing to a retirement account, which is a common context for 
discussing IRAs.
2. **IRA**: The email explicitly mentions "IRA," indicating that the sender is interested in learning more about 
this specific retirement savings vehicle.

Given these points, the email clearly falls under the category of IRA-related inquiries. The other categories (529 
Plan, Cash Management, Youth Account) do not align with the content of the email. 

- **529 Plan**: This is a type of savings plan specifically for education expenses, which is not mentioned in the 
email.
- **Cash Management**: This category typically involves services related to managing day-to-day finances, such as 
checking accounts, savings accounts, and payment services, which are not relevant here.
- **Youth Account**: This category would involve accounts specifically designed for minors, which is also not 
relevant to the email's content.

Therefore, the most appropriate classification is (A) IRA.

In [23]:
# Let's try the same prompt with Anthropic Claude Haiku 3.5.  Again, there is no change to the converse API call.  Only the modelId value changes.

response = bedrock_runtime.converse(
    modelId=inf_profile_claude_haiku_v3_5, # note the modelId change
    messages=zero_shot_messages,
    system=[{"text": zero_shot_system_prompt}],
    inferenceConfig=inference_configs
)

rprint(response['output']['message']['content'][0]['text'])

Classification: (A) IRA

Justification:
The email directly mentions "retirement account" and specifically requests information about "IRA" (Individual 
Retirement Account). The key indicators that lead to this classification are:

1. Direct mention of "IRA" in the email
2. Reference to a "retirement account"
3. Request for resources about IRA
4. The context is clearly about retirement savings

The email is a straightforward inquiry about Individual Retirement Accounts, which makes (A) IRA the most 
appropriate classification. The other options do not match the content of the email:
- (B) 529 Plan is for education savings
- (C) Cash Management relates to banking and cash flow
- (D) Youth Account typically refers to accounts for minors

The sender is seeking information about contributing to and understanding IRAs, which aligns perfectly with the IRA
category.

---

### Few-shot prompting
Giving the model more information about the tasks at hand via examples is called Few-Shot Prompting. It can be used for in-context learning by providing examples of the task and the desired output. We can therefore condition the model on the examples to follow the task guidance more closely.

In [25]:
# If you'd like to try your own prompt, edit this parameter!
few_shot_prompt = """Your job is to generate financial summaries similar to the examples below: 

Microsoft: Microsoft reported record revenue of $51.9 billion in its latest quarterly earnings report, a 20% increase year-over-year. Its net income rose to $16.7 billion, up 21% compared to the previous year. Microsoft's revenue growth was driven by its cloud computing business Azure and Office 365 productivity tools.

Amazon: In its most recent quarterly results, Amazon reported net sales of $96.1 billion, an increase of 15% compared to the same period last year. Amazon's net income decreased to $2.9 billion, down 58% year-over-year, as the company faced rising costs due to inflation and supply chain issues. Amazon Web Services, its cloud computing segment, continued to see strong growth with sales up 33%.

Now provide a brief financial summary for Apple Inc. based on the following key facts:
- Revenue for the last 12 months: $260 billion 
- Net income: $55 billion
- Total assets: $321 billion
- Market capitalization: $2.2 trillion

"""

In [26]:
# We will be using Amazon Nova Micro v1, unless stated otherwise.  But please feel free to try with Haiku or the other models.  
# If you wish to try models other than Amazon Nova Micro v1, Amazon Nova Lite v1 or Anthropic Claude Haiku 3.5, please ensure that you enable them first in the Bedrock console.


few_shot_message = {
    "role": "user",
    "content": [{"text": few_shot_prompt}]
}

few_shot_messages = [few_shot_message]

response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=few_shot_messages,
    inferenceConfig=inference_configs
)

rprint(response['output']['message']['content'][0]['text'])

Apple Inc. reported robust financial performance in the last 12 months, with total revenue reaching $260 billion, 
reflecting a solid growth trajectory. The company's net income stood at $55 billion, showcasing its strong 
profitability. Apple's total assets amounted to $321 billion, underscoring its substantial financial strength and 
resource base. Additionally, the company's market capitalization reached an impressive $2.2 trillion, highlighting 
investor confidence and the market's valuation of Apple's business. These financial metrics collectively indicate 
Apple's continued dominance and resilience in the competitive tech industry.

---

### Chain of Thought prompting
Chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding. The main idea of CoT is that by showing the LLM some few shot exemplars where the reasoning process is explained in the exemplars, the LLM will also show the reasoning process when answering the prompt. This explanation of reasoning often leads to more accurate results.


In [27]:
# If you'd like to try your own prompt, edit this parameter!
cot_prompt = """
Apple Inc. key facts:
- Revenue for the last 12 months: $260 billion 
- Operating Expenses: $205 billion
- Total assets: $321 billion
- Market capitalization: $2.2 trillion

If Apple's revenue is expected to grow by 10% in the next year, however it's expenses are expected to increase by 15%, what would be the expected net income for Apple in the next year?

Think step-by-step and provide your thought process in the <thinking></thinking> XML tags and the final answer in the <answer></answer> XML tags.

"""

In [28]:

cot_message = {
    "role": "user",
    "content": [{"text": cot_prompt}]
}

cot_messages = [cot_message] 

response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=cot_messages,
    inferenceConfig=inference_configs
)

rprint(response['output']['message']['content'][0]['text'])

<thinking>
1. Calculate the expected revenue for the next year:
   - Current revenue: $260 billion
   - Expected growth: 10%
   - Expected revenue = Current revenue * (1 + Growth rate)
   - Expected revenue = $260 billion * (1 + 0.10) = $260 billion * 1.10 = $286 billion

2. Calculate the expected operating expenses for the next year:
   - Current operating expenses: $205 billion
   - Expected increase: 15%
   - Expected operating expenses = Current operating expenses * (1 + Increase rate)
   - Expected operating expenses = $205 billion * (1 + 0.15) = $205 billion * 1.15 = $235.75 billion

3. Calculate the expected net income for the next year:
   - Net income = Expected revenue - Expected operating expenses
   - Expected net income = $286 billion - $235.75 billion = $50.25 billion

</thinking>
<answer>
The expected net income for Apple in the next year would be $50.25 billion.
</answer>

## Prompt Engineering - Different task examples

---

### Conversation Summarization
Prompt engineering for conversation summarization is a process of crafting prompts that guide a language model like Nova in summarizing dialogues or conversations effectively.

In [29]:
summarization_prompt = """
Provide a concise bullet-point summary of the following conversation
<conversation>
Susan: Welcome back, Chuck. This is the second part of our interview, and I want to ask you these questions as quickly as possible because I know you have a flight to catch, so. . . .

Chuck: No problem. I’m happy to chat with you.

Susan: It says on your website that you never studied business before accepting a marketing role at a Spanish-speaking multimedia publisher. Was … was that—

Chuck: Yeah.

Susan: So was that more of a stretch, or … ?

Chuck: Yeah, well, actua—

Susan: Oh, I’m sorry, just a minute. Can everyone else just put themselves on mute, please?

Akim: [inaudible 00:37]

Susan: Great, thanks. OK, so where were we? So, I was asking you if, when you took the new role as Director, it felt like a stretch?

Chuck: Right, yeah. . . . I knew there would be a period of adjustment that I just would have to push through, you know?

Susan: Absolutely. Were there any other challenges or roadblocks that you weren’t expecting though?

Chuck: Hm, roadblocks I wasn’t expecting. Um…

Susan: [laughs] I keep throwing you curveballs.

Chuck: No, it’s OK. So, I actually didn’t anticipate the workload involved in learning a foreign language on the job. It’s like, you know, sometimes. . . . I mean, you need to be really flexible and ready to change it up if your strategy isn’t actually working. Know what I mean?

Susan: [laughs] Definitely. So what was your budget like, I mean, was that, like, a challenge, too?

Chuck: Hoo boy. [laughs] Yeah it definitely was. We had funding available, but we just, um, needed so many hands on deck. It was hard to, you know, actually manage so many people with our existing resources. [coughs]

Susan: I totally understand. So what was your next move? I mean, did you—

Chuck: Yeah, yeah. . . . So next, I kind of wanted to see where I do a gap analysis and throw resources at new initiatives where we weren’t, uh, I mean, I wanted to actually close the loop in those places where we didn’t quite have a foothold.

Susan: Nice, nice.

Chuck: Yeah, we really just needed to keep what was working and then lay out a new plan for [clears throat] future growth.

Susan: So, it looks like we have time for maybe, um, let’s see— maybe one more question.

Chuck: Sounds good. Shoot.

Susan: How did this experience change the way you approached new challenges going forward?

Chuck: Hm, good one. [laughs] I think that I was less, uh, likely to question my own judgement when it came to adaptability. Now I know that I really have it in me, you know?

Really all it takes is a great marketing team underneath you and the freelance resources to put together the best marketing initiative you can afford, you know, with, um, the standard tools high-quality content, good ad placement, good SEO, a strong social presence and then maybe debut a new idea every six months. Um, like, you could come up with a certain theme or catch phrase and actually weave it through all your branding and regular initiatives.

Susan: OK, well thanks, Chuck. This is really good stuff. I know you have to run, um, but I’d just like to thank you for, you know, taking this time with me today.

Chuck: Sure, sure. Of course
</conversation>

"""

In [30]:
summarization_message = {
    "role": "user",
    "content": [{"text": summarization_prompt}]
}

summarization_messages = [summarization_message]

response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=summarization_messages,
    inferenceConfig=inference_configs
)

rprint(response['output']['message']['content'][0]['text'])

- Susan welcomes Chuck back for the second part of their interview, acknowledging his tight schedule.
- Susan asks Chuck about his transition into a marketing role without prior business studies.
- Chuck admits the role was a stretch and involved a period of adjustment.
- Susan inquires about unexpected challenges; Chuck mentions the difficulty of learning a new language and managing
a large workload.
- Susan asks about budget challenges; Chuck confirms it was difficult to manage resources effectively.
- Chuck discusses his strategy to conduct a gap analysis and focus on new initiatives for growth.
- Susan asks how the experience changed Chuck's approach to future challenges.
- Chuck reflects on increased confidence in adaptability and emphasizes the importance of a strong marketing team 
and strategic planning.
- Susan thanks Chuck for his time and insights before concluding the interview.

### Question & Answering
Prompt engineering for question answering is a process of crafting prompts that guide a language model like Nova in answering questions on provided context effectively. This is used as a basis for more advanced techniques such as Retrieval-Augmented Generation (RAG).

In [31]:
qa_system_prompt = """You are a customer service agent tasked with answering questions about various financial products and services. Please output your answer and then justify your classification. """

qa_prompt = """
<context>
When the teen turns 18, their account is eligible to transition to the regular Fidelity brokerage account with expanded features like option and margin trading.
The teen will be prompted to transition their account starting on their 18th birthday. They will have 60 days to do so before their debit card and ability to trade will be restricted. Once the account is transitioned, the debit card that the teen was issued for their Fidelity Youth™ Account will continue to be valid until it expires; at that point, a new debit card will be issued.
Once a teen turns 18, the teen may choose whether or not the parent/guardian will continue to have access to that teen’s Fidelity Youth Account information. Teens can still use the Fidelity Youth™ app when they turn 18, however additional capabilities are available to them in the Fidelity mobile app.
</context> 

<question> What is the role of a parent/ guardian according to Fidelity youth accounts? </question>

"""


In [32]:
qa_message = {
    "role": "user",
    "content": [{"text": qa_prompt}]
}

qa_messages = [qa_message]

response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=messages,
    system=[{"text": qa_system_prompt}],
    inferenceConfig=inference_configs
)

rprint(response['output']['message']['content'][0]['text'])

One of the best ways to save for your children's college tuition is to open a 529 College Savings Plan. This is a 
tax-advantaged savings plan designed specifically for education expenses. Contributions to a 529 plan grow 
tax-free, and withdrawals for qualified education expenses are also tax-free. Here are some steps to consider:

1. **Research Different Plans**: There are two main types of 529 plans: prepaid tuition plans and education savings
plans. Education savings plans, often called 529 college savings plans, are more common and offer investment 
options similar to a mutual fund.

2. **Compare Fees and Investment Options**: Different 529 plans have varying fees and investment options. It's 
important to compare these to find a plan that fits your financial goals and risk tolerance.

3. **Start Early**: The earlier you start saving, the more time your money has to grow. Even small contributions 
can add up significantly over time due to compound interest.

4. **Consistent Contributions**: Make regular contributions to your 529 plan. Many plans offer automatic 
contribution options that can help you stay consistent.

5. **Consult a Financial Advisor**: If you're unsure about the best plan for your situation, consider consulting a 
financial advisor who can provide personalized advice.

**Justification for Classification**: This response is classified as providing detailed, actionable advice on a 
specific financial product (529 College Savings Plan) tailored to the customer's need to save for college tuition. 
It includes steps to take, types of plans available, and considerations to keep in mind, making it a comprehensive 
answer.

### Content Generation
We can use prompt engineering to guide the model in generating content that is relevant to the task at hand. This can be used for generating content for a variety of tasks such as content creation, content summarization, and more.
In this example, the model will generate a table using provided data and as well as some questions that it can answer based on this data. This approach is common for getting the model to address more complex research type tasks where multiple steps are involved in generating the final output.

In [33]:
content_generation_prompt = """Below is research that you performed previously. On basis of this research please answer the question below. 
If it does not include the exact answer, please respond with relevant information from the research in the response. Prefer more recent sources when possible.
If the context contains publication ids, please include citations in the response. Citations should be formatted as [publication_id] or [org_id]
If the user asks for a chart, return only machine readable json structure for the chart in HighCharts format without any formatting. 
Prefer answering in First Person (I, me, my) style.
In your response, after you answer draw a line, and include 3 additional questions you can answer with the research, one question per line.
Example:
{Your Answer}

Additional questions I can answer::
Question 1
Question 2
Question 3

<research>
JPMorgan Chase Bank, N.A. (164500) Financials: 
 
|Indicator|2020(annual)|2021(annual)|2022(annual)|
|---|---|---|---|
|Total assets (USD Billion)|3025.28|3306.98|3201.94|
|Tangible Common Equity (USD Billion)|221.41|262.33|280.96|
|Problem Loans / Gross Loans|1.64|1.14|1.00|
|Tangible Common Equity / Risk Weighted Assets|16.48|18.83|19.04|
|Problem Loans / (Tangible Common Equity + Loan Loss Reserve)|6.71|4.44|3.80|
|Net Interest Margin|2.13|1.74|2.22|
|PPI / Average RWA|3.30|2.90|3.45|
|Net Income / Tangible Assets|0.70|1.16|1.09|
|Cost / Income Ratio|58.76|61.90|57.71|
|Market Funds / Tangible Banking Assets|15.38|12.38|12.43|
|Liquid Banking Assets / Tangible Banking Assets|55.38|55.31|49.91|
|Gross Loans / Due to Customers|47.24|44.01|48.72|
 
Bank of America Corporation (541000) Financials: 
 
|Indicator|2020(annual)|2021(annual)|2022(annual)|
|---|---|---|---|
|Total assets (USD Billion)|2786.01|3146.17|3024.42|
|Tangible Common Equity (USD Billion)|174.68|175.58|190.40|
|Problem Loans / Gross Loans|1.18|0.95|0.91|
|Tangible Common Equity / Risk Weighted Assets|12.74|12.55|13.49|
|Problem Loans / (Tangible Common Equity + Loan Loss Reserve)|5.92|5.26|4.87|
|Net Interest Margin|1.75|1.52|1.82|
|PPI / Average RWA|1.94|1.96|2.23|
|Net Income / Tangible Assets|0.67|0.92|0.71|
|Cost / Income Ratio|67.23|69.57|66.12|
|Market Funds / Tangible Banking Assets|21.09|21.26|22.71|
|Liquid Banking Assets / Tangible Banking Assets|57.50|58.98|55.10|
|Gross Loans / Due to Customers|54.20|50.35|56.05|
 
Wells Fargo Bank, N.A. (811500) Financials: 
 
|Indicator|2020(annual)|2021(annual)|2022(annual)|
|---|---|---|---|
|Total assets (USD Billion)|1767.81|1779.50|1717.53|
|Tangible Common Equity (USD Billion)|145.47|148.21|149.95|
|Problem Loans / Gross Loans|1.69|1.36|1.14|
|Tangible Common Equity / Risk Weighted Assets|14.36|15.35|15.34|
|Problem Loans / (Tangible Common Equity + Loan Loss Reserve)|9.18|7.37|6.45|
|Net Interest Margin|2.41|2.13|2.77|
|PPI / Average RWA|1.46|1.91|2.22|
|Net Income / Tangible Assets|0.20|1.00|0.95|
|Cost / Income Ratio|75.75|69.83|67.73|
|Market Funds / Tangible Banking Assets|5.58|3.44|6.68|
|Liquid Banking Assets / Tangible Banking Assets|41.44|41.29|35.80|
|Gross Loans / Due to Customers|61.22|57.78|65.53|
 
</research>
 
<analysis>
 
(Doc Id: PBC_1362054, Publish Date: 2023-05-11, Title:Bank of America Corporation: Update to credit analysis following ratings upgrade)
 
1. Support and Structural Considerations:
About Moody's Bank ScorecardOur Bank Scorecard is designed to capture, express and explain in summary form our Rating Committee's judgment. When read in conjunction with our research, a fulsome presentation of our judgment is expressed. As a result, the output of our Scorecard may materially differ from that suggested by raw data alone (though it has been calibrated to avoid the frequent need for strong divergence). The Scorecard output and the individual scores are discussed in rating committees and may be adjusted up or down to reflect conditions specific to each rated entity. As per Moody's Banks rating methodology, the historic ratios in the scorecard for Capital are as of most recent period, for Asset Risk and Profitability they are the worse of the most recent year-to-date period or the average of the last three years and the most recent year-to-date, and for Funding Structure and Liquid Resources they are as of the most recent year-end.
2. RATINGS:
Please see the ratings section at the end of this report for more information. The ratings and outlook shown reflect information as of the publication date. 
Summary:
Bank of America Corporation (BAC, A1 stable) is the parent holding company for the second largest banking group in the US. The group is a global systemically important bank operating primarily through its principal bank subsidiary, Bank of America N.A. (BANA, Aa1 stable, a2 baseline credit assessment). BAC's credit profile is supported by its conservative risk appetite, a balanced and diversified business mix, strong funding and liquidity, robust cost discipline, strengthened capital and resilient profitability. During 2022, in response to increases in its regulatory capital requirements, BAC boosted its capital ratios significantly.
 
</analysis>
 
<questions>
1. Can you compare the financials for JPMC, Bank of America and Wells fargo in a table format?
2. For statements referenced from documents include a reference in the format [docId].
DO NOT UNDER ANY CIRCUMSTANCES USE ANY DATA OTHER THAN MENTIONED IN THE RESEARCH SECTION.
</questions>

"""

In [34]:
content_generation_message = {
    "role": "user",
    "content": [{"text": content_generation_prompt}]
}
content_generation_messages = [content_generation_message]

response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=content_generation_messages,
    inferenceConfig=inference_configs
)

rprint(Markdown(response['output']['message']['content'][0]['text']))

Here is a comparison of the financials for JPMorgan Chase Bank, N.A., Bank of America Corporation, and Wells Fargo 
Bank, N.A. in a table format:                                                                                      

                                                                                                          
  Indicator                                                      JPMC 2022   BAC 2022   Wells Fargo 2022  
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  Total assets (USD Billion)                                     3201.94     3024.42    1717.53           
  Tangible Common Equity (USD Billion)                           280.96      190.40     149.95            
  Problem Loans / Gross Loans                                    1.00        0.91       1.14              
  Tangible Common Equity / Risk Weighted Assets                  19.04       13.49      15.34             
  Problem Loans / (Tangible Common Equity + Loan Loss Reserve)   3.80        4.87       6.45              
  Net Interest Margin                                            2.22        1.82       2.77              
  Net Income / Tangible Assets                                   1.09        0.71       0.95              
  Cost / Income Ratio                                            57.71       66.12      67.73             
                                                                                                          

BAC's credit profile is supported by its conservative risk appetite, a balanced and diversified business mix,      
strong funding and liquidity, robust cost discipline, strengthened capital and resilient profitability             
[PBC_1362054].                                                                                                     

Here is the JSON structure for the chart in HighCharts format:                                                     

                                                                                                                   
 {                                                                                                                 
   "chart": {                                                                                                      
     "type": "column"                                                                                              
   },                                                                                                              
   "title": {                                                                                                      
     "text": "Comparison of Financials for JPMC, BAC, and Wells Fargo"                                             
   },                                                                                                              
   "xAxis": {                                                                                                      
     "categories": ["JPMC", "BAC", "Wells Fargo"]                                                                  
   },                                                                                                              
   "yAxis": {                                                                                                      
     "title": {                                                                                                    
       "text": "Amount (USD Billion)"                                                                              
     }                                                                                                             
   },                                                                                                              
   "plotOptions": {                                                                                                
     "column": {                            

### Multi-modal prompts
In addition to text, some models allow for text and vision prompt capability.

This is useful in a number of use cases including:
- Extracting information from charts, graphs, diagrams, etc
- Summarizing rich content
- Answering questions based on images
- Being able to effective handle more complex format types. For example we can ask the model to narrate a PowerPoint presentation and then utilize the narrative in downstream semantic search applications

We will now be using Amazon Nova Lite, which allows for text and vision inputs.  Amazon Nova Micro is for text only.

Let's look at an example using a page out of [JPMC's 2022 Annual Report](data/jpmc_annual_report_page_6.pdf)

In [35]:
# Let's take a look at the pdf document
# You can also find this document directly under "fsi-genai-bootcamp/02_prompt_engineering/data/"
# The code in this cell is just used to view the document, it is not used in the Bedrock prompt

pdf_path = "data/jpmc_annual_report_page_6.pdf"
from base64 import b64encode
from IPython.display import display, HTML

def display_pdf(pdf_path, width=1000, height=700):
    with open(pdf_path, 'rb') as f:
        pdf_bytes = f.read()
    base64_pdf = b64encode(pdf_bytes).decode('utf-8')
    display(HTML(f'<embed src="data:application/pdf;base64,{base64_pdf}" width="{width}" height="{height}" type="application/pdf"/>'))


display_pdf(pdf_path)

In [36]:
prompt_data_text = """
Extract the data contained in the chart and provide it in a json format. The data should include the Net Income, Diluted EPS, and ROTCE from 2004 to 2022 and should be structured as follows:
{"net_income":{"units:"$B", "data_points":{"2004": 4.5}}, "diluted_eps":{...}, "rotce":{...}}

"""

In [37]:
def read_pdf_as_bytes(pdf_path):
    """Reads a PDF file and returns its content as bytes."""
    with open(pdf_path, "rb") as pdf_file:
        pdf_bytes = pdf_file.read()
    return pdf_bytes


pdf_path = "data/jpmc_annual_report_page_6.pdf"

pdf_content = read_pdf_as_bytes(pdf_path)

In [38]:
# note how the message has a text and an document component
# Amazon Nova multi-modal models can read text, documents and images
# Some multi-modal models have image reading capability only, and documents such as pdf will need to be converted into images beforehand
# E.g., the below message input will not work with Anthropic Claude 3.5, unless the pdf is first converted into image format

pdf_message = {
    "role": "user",
    "content": [
        {"text": prompt_data_text},
        {
            "document": {
                "format": "pdf",
                "name": "jpmc_annual_rpt",
                "source": {"bytes": pdf_content},
            }
        },
    ],
}

pdf_messages = [pdf_message]

response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=pdf_messages,
    inferenceConfig=inference_configs,
)

rprint(response)
rprint(Markdown(response['output']['message']['content'][0]['text']))

{
    'ResponseMetadata': {
        'RequestId': 'cf8818b6-9946-4529-a84a-59b734b2f16f',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'date': 'Mon, 12 May 2025 09:26:04 GMT',
            'content-type': 'application/json',
            'content-length': '1749',
            'connection': 'keep-alive',
            'x-amzn-requestid': 'cf8818b6-9946-4529-a84a-59b734b2f16f'
        },
        'RetryAttempts': 0
    },
    'output': {
        'message': {
            'role': 'assistant',
            'content': [
                {
                    'text': '```json\n{\n  "net_income": {\n    "units": "$B",\n    "data_points": {\n      "2004":
1.52,\n      "2005": 4.5,\n      "2006": 2.35,\n      "2007": 8.5,\n      "2008": 14.4,\n      "2009": 11.7,\n     
"2010": 17.4,\n      "2011": 19.0,\n      "2012": 15.0,\n      "2013": 17.9,\n      "2014": 5.29,\n      "2015": 
6.00,\n      "2016": 6.19,\n      "2017": 6.31,\n      "2018": 9.00,\n      "2019": 10.72,\n      "2020": 36.4,\n  
"2021": 38.4,\n      "2022": 37.7\n    }\n  },\n  "diluted_eps": {\n    "units": "$",\n    "data_points": {\n      
"2004": 1.52,\n      "2005": 2.35,\n      "2006": 2.35,\n      "2007": 8.5,\n      "2008": 14.4,\n      "2009": 
11.7,\n      "2010": 15.0,\n      "2011": 15.0,\n      "2012": 15.0,\n      "2013": 11.0,\n      "2014": 13.0,\n   
"2015": 13.0,\n      "2016": 13.0,\n      "2017": 12.0,\n      "2018": 17.0,\n      "2019": 19.0,\n      "2020": 
19.3,\n      "2021": 18.0,\n      "2022": 18.5\n    }\n  },\n  "rotce": {\n    "units": "%",\n    "data_points": 
{\n      "2004": 10.0,\n      "2005": 15.0,\n      "2006": 15.0,\n      "2007": 6.0,\n      "2008": 22.0,\n      
"2009": 10.0,\n      "2010": 15.0,\n      "2011": 15.0,\n      "2012": 15.0,\n      "2013": 11.0,\n      "2014": 
13.0,\n      "2015": 13.0,\n      "2016": 13.0,\n      "2017": 12.0,\n      "2018": 17.0,\n      "2019": 19.0,\n   
"2020": 14.0,\n      "2021": 23.0,\n      "2022": 18.0\n    }\n  }\n}\n```'
                }
            ]
        }
    },
    'stopReason': 'end_turn',
    'usage': {'inputTokens': 2175, 'outputTokens': 818, 'totalTokens': 2993},
    'metrics': {'latencyMs': 8179}
}

                                                                                                                   
 {                                                                                                                 
   "net_income": {                                                                                                 
     "units": "$B",                                                                                                
     "data_points": {                                                                                              
       "2004": 1.52,                                                                                               
       "2005": 4.5,                                                                                                
       "2006": 2.35,                                                                                               
       "2007": 8.5,                                                                                                
       "2008": 14.4,                                                                                               
       "2009": 11.7,                                                                                               
       "2010": 17.4,                                                                                               
       "2011": 19.0,                                                                                               
       "2012": 15.0,                                                                                               
       "2013": 17.9,                                                                                               
       "2014": 5.29,                                                                                               
       "2015": 6.00,                                                                                               
       "2016": 6.19,                                                                                               
       "2017": 6.31,                                                                                               
       "2018": 9.00,                                                                                               
       "2019": 10.72,                                                                                              
       "2020": 36.4,                                                                                               
       "2021": 38.4,                                                                                               
       "2022": 37.7                                                                                                
     }                                                                                                             
   },                                                                                                              
   "diluted_eps": {                                                                                                
     "units": "$",                                                                                                 
     "data_points": {                                                                                              
       "2004": 1.52,                                                                                               
       "2005": 2.35,                                                                                               
       "2006": 2.35,                                                                                               
       "2007": 8.5,                                                                                                
       "2008": 14.4,                                                                                               
       "2009": 11.7,                                    

## Conclusion

We are just getting warmed up, we explored basic prompting techiques. Please proceed to the next Notebook.